In [13]:
import pandas as pd
import numpy as np

import os
import sys

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from tensorflow.keras.datasets import fashion_mnist
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Activation
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner.tuners import RandomSearch

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [14]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
import matplotlib.pyplot as plt
%matplotlib inline
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [15]:
import time
LOG_DIR = f"{int(time.time())}"

In [16]:
def build_model(hp):  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()
    
    model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten()) 

    model.add(Dense(10))
    model.add(Activation("softmax"))

    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model


In [17]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=1,  # how many model variations to test?
    executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)


In [19]:
tuner.search(x=x_train,
             y=y_train,
             verbose=2, # just slapping this here bc jupyter notebook. The console out was getting messy.
             epochs=1,
             batch_size=64,
             #callbacks=[tensorboard],  # if you have callbacks like tensorboard, they go here.
             validation_data=(x_test, y_test))


Trial 1 Complete [00h 00m 32s]
val_accuracy: 0.8317000269889832

Best val_accuracy So Far: 0.8317000269889832
Total elapsed time: 00h 00m 32s
INFO:tensorflow:Oracle triggered exit


In [21]:
def build_model2(hp):  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()
    
    model.add(Conv2D(hp.Int('input_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3), input_shape=x_train.shape[1:]))
    
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of layers.
        model.add(Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=256,
                                step=32), (3, 3)))
        model.add(Activation('relu'))

    model.add(Flatten()) 
    model.add(Dense(10))
    model.add(Activation("softmax"))

    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model


In [24]:
tuner = RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=1,  # how many model variations to test?
    executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)

tuner.search(x=x_train,
             y=y_train,
             verbose=2, # just slapping this here bc jupyter notebook. The console out was getting messy.
             epochs=1,
             batch_size=64,
             #callbacks=[tensorboard],  # if you have callbacks like tensorboard, they go here.
             validation_data=(x_test, y_test))


Trial 1 Complete [00h 03m 42s]
val_accuracy: 0.862500011920929

Best val_accuracy So Far: 0.862500011920929
Total elapsed time: 00h 03m 42s
INFO:tensorflow:Oracle triggered exit


In [25]:
tuner.get_best_hyperparameters()[0].values


{'input_units': 160,
 'n_layers': 3,
 'conv_0_units': 96,
 'conv_1_units': 32,
 'conv_2_units': 32}

In [26]:
tuner.get_best_models()[0].summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 160)       1600      
_________________________________________________________________
activation (Activation)      (None, 26, 26, 160)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 160)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 96)        138336    
_________________________________________________________________
activation_1 (Activation)    (None, 11, 11, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 32)          27680     
_________________________________________________________________
activation_2 (Activation)    (None, 9, 9, 32)          0